# Preprocessing

In [54]:
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/onurergun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
Finaldata = pd.DataFrame(columns=['animeTitle','animeDescription'])
for i in range(1,50):
    anime_tsv = open('tsv_files/anime_'+str(i)+'.tsv', 'r',encoding="utf8")
    data=pd.read_table(anime_tsv)[['animeTitle','animeDescription']]
    Finaldata=pd.merge(Finaldata,data,how='outer')
    anime_tsv.close()

In [56]:
keywords = []
words=[]
for i in range(len(Finaldata.iloc[:,1])):
    query=Finaldata.iloc[i,1]
    
    # lowercase all the query
    query = query.lower()
    
    #Remove punctuation
    token = RegexpTokenizer(r'\w+')
    query=token.tokenize(query)
    
    #Remove stopwords
    query = [word for word in query if not word in stopwords.words()]
    
    #STEMMING using Snowball
    stemmer = SnowballStemmer("english")
    
    #we add every word stemmed to a list we created
    words.append([stemmer.stem(word) for word in query])
    for word in query:
        keywords.append(stemmer.stem(word))

In [51]:
# Take words only one time
keywords = set(keywords)

In [6]:
# Assign an integer to each different words
voc = {}
i=1
for word in keywords:
    voc.update({word:i})
    i+=1

In [7]:
# Assign to each integer value (the word's associated value) a list of document's names that contains that word
dic={}
for q in voc:
    l = []
    for i in range(len(words)):
        if q in words[i]:
            l.append('anime_'+str(i+1))
    dic.update({voc[q]:l})

In [10]:
n=list(input().split(' '))
inp=[]

 Human Fight


In [11]:
#we steam the words in input
for w in n:
    stemmer = SnowballStemmer("english")
    w=stemmer.stem(w)
    inp.append(w)
print(inp)

['human', 'fight']


In [12]:
#inp = ['alchem', 'alchemist']

In [13]:
# Creates a list of list in which each sublist contains all the documents in which each word given appears
documents = []
for elem in inp:
    if elem in voc:
        documents.append(dic[voc[elem]])

# Confront all the words in the first sublist with all the other sublist and gives only the documents that contains
# all the words given in input

anime=[]                                              # Here we'll store the documents that contains all the words
for i in range(len(documents[0])):                    # for cicle to scroll over the elements of first sublist
    for j in range(1,len(documents)):                 # for cicle to check all other sublists
        l = 0                                         # counter used to check if a word is in all the lists
        if documents[0][i] in documents[j]:           # condition that if satisfied increase the counter
            l += 1
    if l == len(documents)-1:                         # condition that if satisfied append the document we are testing to list
        anime.append(documents[0][i])
            
print(anime) 

['anime_8', 'anime_27']


In [15]:
# Read url text file that we need in the final part
url = open('links.txt',encoding = 'utf8')
urls = url.readlines()
url.close()

In [16]:
# print all the information about all the anime that match all the key words
for elem in anime:
    n = int(''.join(filter(str.isdigit, elem)))-1       # take the number of each anime file and subtract 1 to get the right row
    print(Finaldata.iloc[n,0])                          # print the name of anime in row n
    print(Finaldata.iloc[n,1])                          # print the description of anime in row n
    print(urls[n][0:-1])                                # print the url (we stopped at -1 to avoid no-url symbols)

Fullmetal Alchemist: Brotherhood
After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.The brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.As 

In [18]:
Finaldata['Url'] = urls[0:49] # modify urls  when we use all the dataset

In [19]:
finded = []                                           # Creates a void list
for elem in anime:                                    # iterates through all intersect anime
    n = int(''.join(filter(str.isdigit, elem)))-1     # find the number in the document anime
    finded.append(n)                                  # append to the void list each of the values finded

Finaldata.iloc[finded]                                # returns only the rows that have matched

,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/5114/Fullmetal_A...


In [20]:
# Exercise 2.2

In [21]:
from collections import Counter
import math

In [22]:
es22 = list()
for i in range(len(words)):
    c = Counter(words[i])
    es22.append(c)

In [23]:
D = len(words)
idf = []
for w in keywords:
    l=0
    for doc in words:
        if w in doc:
            l+=1
    idf.append(math.log10(D/l))

In [24]:
for c in es22:
    for word in c:
            i = 0
            c[word] = round(c[word]/len(c) * idf[i],2)
            i+=1

In [25]:
new = dict()
for i in range(len(words)):
    for word in words[i]:
        if word in new:
            new[word].append(['anime_'+str(i+1),es22[word]])
        else:
            new.update({voc[word]:[['anime_'+str(i+1),es22[i][word]]]})

In [26]:
new

{487: [['anime_1', 0.02]],
 1205: [['anime_1', 0.02]],
 399: [['anime_40', 0.02]],
 140: [['anime_28', 0.02]],
 1555: [['anime_38', 0.03]],
 1438: [['anime_1', 0.03]],
 98: [['anime_1', 0.02]],
 374: [['anime_1', 0.05]],
 16: [['anime_33', 0.02]],
 1170: [['anime_1', 0.05]],
 805: [['anime_48', 0.03]],
 1699: [['anime_1', 0.02]],
 1686: [['anime_45', 0.02]],
 621: [['anime_46', 0.02]],
 1176: [['anime_1', 0.02]],
 846: [['anime_1', 0.02]],
 205: [['anime_1', 0.02]],
 1730: [['anime_1', 0.02]],
 1591: [['anime_43', 0.05]],
 1853: [['anime_1', 0.02]],
 1840: [['anime_46', 0.02]],
 665: [['anime_48', 0.06]],
 1103: [['anime_18', 0.03]],
 945: [['anime_28', 0.02]],
 1372: [['anime_1', 0.02]],
 1146: [['anime_46', 0.02]],
 439: [['anime_48', 0.06]],
 125: [['anime_41', 0.02]],
 28: [['anime_36', 0.02]],
 1817: [['anime_48', 0.03]],
 119: [['anime_20', 0.02]],
 136: [['anime_23', 0.05]],
 1327: [['anime_33', 0.02]],
 1304: [['anime_21', 0.03]],
 1184: [['anime_1', 0.02]],
 549: [['anime_29',